In [14]:
import re

# Input assembly listing
assembly = """
8000018c <test_2>:
8000018c:	00200193          	li	gp,2
80000190:	ff0105b7          	lui	a1,0xff010
80000194:	f0058593          	addi	a1,a1,-256 # ff00ff00 <_end+0x7f00df00>
80000198:	0f0f1637          	lui	a2,0xf0f1
8000019c:	f0f60613          	addi	a2,a2,-241 # f0f0f0f <_start-0x70f0f0f1>
800001a0:	00c5f733          	and	a4,a1,a2
800001a4:	0f0013b7          	lui	t2,0xf001
800001a8:	f0038393          	addi	t2,t2,-256 # f000f00 <_start-0x70fff100>
800001ac:	48771c63          	bne	a4,t2,80000644 <fail>

800001b0 <test_3>:
800001b0:	00300193          	li	gp,3
800001b4:	0ff015b7          	lui	a1,0xff01
800001b8:	ff058593          	addi	a1,a1,-16 # ff00ff0 <_start-0x700ff010>
800001bc:	f0f0f637          	lui	a2,0xf0f0f
800001c0:	0f060613          	addi	a2,a2,240 # f0f0f0f0 <_end+0x70f0d0f0>
800001c4:	00c5f733          	and	a4,a1,a2
800001c8:	00f003b7          	lui	t2,0xf00
800001cc:	0f038393          	addi	t2,t2,240 # f000f0 <_start-0x7f0fff10>
800001d0:	46771a63          	bne	a4,t2,80000644 <fail>

800001d4 <test_4>:
800001d4:	00400193          	li	gp,4
800001d8:	00ff05b7          	lui	a1,0xff0
800001dc:	0ff58593          	addi	a1,a1,255 # ff00ff <_start-0x7f00ff01>
800001e0:	0f0f1637          	lui	a2,0xf0f1
800001e4:	f0f60613          	addi	a2,a2,-241 # f0f0f0f <_start-0x70f0f0f1>
800001e8:	00c5f733          	and	a4,a1,a2
800001ec:	000f03b7          	lui	t2,0xf0
800001f0:	00f38393          	addi	t2,t2,15 # f000f <_start-0x7ff0fff1>
800001f4:	44771863          	bne	a4,t2,80000644 <fail>

800001f8 <test_5>:
800001f8:	00500193          	li	gp,5
800001fc:	f00ff5b7          	lui	a1,0xf00ff
80000200:	00f58593          	addi	a1,a1,15 # f00ff00f <_end+0x700fd00f>
80000204:	f0f0f637          	lui	a2,0xf0f0f
80000208:	0f060613          	addi	a2,a2,240 # f0f0f0f0 <_end+0x70f0d0f0>
8000020c:	00c5f733          	and	a4,a1,a2
80000210:	f000f3b7          	lui	t2,0xf000f
80000214:	42771863          	bne	a4,t2,80000644 <fail>

80000218 <test_6>:
80000218:	00600193          	li	gp,6
8000021c:	ff0105b7          	lui	a1,0xff010
80000220:	f0058593          	addi	a1,a1,-256 # ff00ff00 <_end+0x7f00df00>
80000224:	0f0f1637          	lui	a2,0xf0f1
80000228:	f0f60613          	addi	a2,a2,-241 # f0f0f0f <_start-0x70f0f0f1>
8000022c:	00c5f5b3          	and	a1,a1,a2
80000230:	0f0013b7          	lui	t2,0xf001
80000234:	f0038393          	addi	t2,t2,-256 # f000f00 <_start-0x70fff100>
80000238:	40759663          	bne	a1,t2,80000644 <fail>

8000023c <test_7>:
8000023c:	00700193          	li	gp,7
80000240:	0ff015b7          	lui	a1,0xff01
80000244:	ff058593          	addi	a1,a1,-16 # ff00ff0 <_start-0x700ff010>
80000248:	f0f0f637          	lui	a2,0xf0f0f
8000024c:	0f060613          	addi	a2,a2,240 # f0f0f0f0 <_end+0x70f0d0f0>
80000250:	00c5f633          	and	a2,a1,a2
80000254:	00f003b7          	lui	t2,0xf00
80000258:	0f038393          	addi	t2,t2,240 # f000f0 <_start-0x7f0fff10>
8000025c:	3e761463          	bne	a2,t2,80000644 <fail>

80000260 <test_8>:
80000260:	00800193          	li	gp,8
80000264:	ff0105b7          	lui	a1,0xff010
80000268:	f0058593          	addi	a1,a1,-256 # ff00ff00 <_end+0x7f00df00>
8000026c:	00b5f5b3          	and	a1,a1,a1
80000270:	ff0103b7          	lui	t2,0xff010
80000274:	f0038393          	addi	t2,t2,-256 # ff00ff00 <_end+0x7f00df00>
80000278:	3c759663          	bne	a1,t2,80000644 <fail>
"""

# Regex pattern to match each function and instruction
pattern = re.compile(r"(\S+)\s*<(\S+)>:\s*([\s\S]+?)\n\n", re.DOTALL)

# Parse the assembly into the desired format
functions = []

matches = pattern.findall(assembly)

for match in matches:
    address = match[0]
    function_name = match[1]
    instructions = match[2].strip().splitlines()
    
    func_str = f"var {function_name} : ArrayBuffer[String] = ArrayBuffer("
    instruction_strs = [f"\"{line.strip().split(':')[1].strip()}\"" for line in instructions]
    func_str += ", ".join(instruction_strs)
    func_str += ")"
    
    functions.append(func_str)

# Output the result
def remove_addresses(lines):
    return [re.sub(r'^\S+\s+', '', line) for line in lines]

for func in functions:
  print(func)

var test_2 : ArrayBuffer[String] = ArrayBuffer("00200193          	li	gp,2", "ff0105b7          	lui	a1,0xff010", "f0058593          	addi	a1,a1,-256 # ff00ff00 <_end+0x7f00df00>", "0f0f1637          	lui	a2,0xf0f1", "f0f60613          	addi	a2,a2,-241 # f0f0f0f <_start-0x70f0f0f1>", "00c5f733          	and	a4,a1,a2", "0f0013b7          	lui	t2,0xf001", "f0038393          	addi	t2,t2,-256 # f000f00 <_start-0x70fff100>", "48771c63          	bne	a4,t2,80000644 <fail>")
var test_3 : ArrayBuffer[String] = ArrayBuffer("00300193          	li	gp,3", "0ff015b7          	lui	a1,0xff01", "ff058593          	addi	a1,a1,-16 # ff00ff0 <_start-0x700ff010>", "f0f0f637          	lui	a2,0xf0f0f", "0f060613          	addi	a2,a2,240 # f0f0f0f0 <_end+0x70f0d0f0>", "00c5f733          	and	a4,a1,a2", "00f003b7          	lui	t2,0xf00", "0f038393          	addi	t2,t2,240 # f000f0 <_start-0x7f0fff10>", "46771a63          	bne	a4,t2,80000644 <fail>")
var test_4 : ArrayBuffer[String] = ArrayBuffer("00400193     

In [1]:
import re

# Input assembly listing
assembly = """
8000018c <test_2>:
8000018c:  00200193           li    gp,2
80000190:  00000593           li    a1,0
80000194:  00000613           li    a2,0
80000198:  00c58733           add   a4,a1,a2
8000019c:  00000393           li    t2,0
800001a0:  4c771663           bne   a4,t2,8000066c <fail>

800001a4 <test_3>:
800001a4:  00300193           li    gp,3
800001a8:  00100593           li    a1,1
800001ac:  00100613           li    a2,1
800001b0:  00c58733           add   a4,a1,a2
800001b4:  00200393           li    t2,2
800001b8:  4a771a63           bne   a4,t2,8000066c <fail>

800001bc <test_4>:
800001bc:  00400193           li    gp,4
800001c0:  00300593           li    a1,3
800001c4:  00700613           li    a2,7
800001c8:  00c58733           add   a4,a1,a2
800001cc:  00a00393           li    t2,10
800001d0:  48771e63           bne   a4,t2,8000066c <fail>

800001d4 <test_5>:
800001d4:  00500193           li    gp,5
800001d8:  00000593           li    a1,0
800001dc:  ffff8637           lui   a2,0xffff8
800001e0:  00c58733           add   a4,a1,a2
800001e4:  ffff83b7           lui   t2,0xffff8
800001e8:  48771263           bne   a4,t2,8000066c <fail>
"""

# Regex pattern to match each function and instruction
pattern = re.compile(r"(\S+)\s*<(\S+)>:\s*([\s\S]+?)\n\n", re.DOTALL)

# Parse the assembly into the desired format
functions = []

matches = pattern.findall(assembly)

# Function to remove addresses from instructions
def remove_addresses(lines):
    return [re.sub(r'^\S+\s+', '', line) for line in lines]

for match in matches:
    address = match[0]
    function_name = match[1]
    instructions = match[2].strip().splitlines()

    # Clean the instructions to remove the address (addresses are the first part of each line)
    cleaned_instructions = remove_addresses(instructions)

    # Build the function string
    func_str = f"var {function_name} : List[String] = List("
    instruction_strs = [f"\"{line.strip()}\"" for line in cleaned_instructions]
    func_str += ", ".join(instruction_strs)
    func_str += ")"

    functions.append(func_str)

# Output the result
for func in functions:
    print(func)


var test_2 : List[String] = List("00200193           li    gp,2", "00000593           li    a1,0", "00000613           li    a2,0", "00c58733           add   a4,a1,a2", "00000393           li    t2,0", "4c771663           bne   a4,t2,8000066c <fail>")
var test_3 : List[String] = List("00300193           li    gp,3", "00100593           li    a1,1", "00100613           li    a2,1", "00c58733           add   a4,a1,a2", "00200393           li    t2,2", "4a771a63           bne   a4,t2,8000066c <fail>")
var test_4 : List[String] = List("00400193           li    gp,4", "00300593           li    a1,3", "00700613           li    a2,7", "00c58733           add   a4,a1,a2", "00a00393           li    t2,10", "48771e63           bne   a4,t2,8000066c <fail>")


In [3]:
import re

# Input assembly listing
assembly = """
8000018c <test_2>:
8000018c:  00200193           li    gp,2
80000190:  00000593           li    a1,0
80000194:  00000613           li    a2,0
80000198:  00c58733           add   a4,a1,a2
8000019c:  00000393           li    t2,0
800001a0:  4c771663           bne   a4,t2,8000066c <fail>

800001a4 <test_3>:
800001a4:  00300193           li    gp,3
800001a8:  00100593           li    a1,1
800001ac:  00100613           li    a2,1
800001b0:  00c58733           add   a4,a1,a2
800001b4:  00200393           li    t2,2
800001b8:  4a771a63           bne   a4,t2,8000066c <fail>

800001bc <test_4>:
800001bc:  00400193           li    gp,4
800001c0:  00300593           li    a1,3
800001c4:  00700613           li    a2,7
800001c8:  00c58733           add   a4,a1,a2
800001cc:  00a00393           li    t2,10
800001d0:  48771e63           bne   a4,t2,8000066c <fail>

800001d4 <test_5>:
800001d4:  00500193           li    gp,5
800001d8:  00000593           li    a1,0
800001dc:  ffff8637           lui   a2,0xffff8
800001e0:  00c58733           add   a4,a1,a2
800001e4:  ffff83b7           lui   t2,0xffff8
800001e8:  48771263           bne   a4,t2,8000066c <fail>
"""

# Regex pattern to match each function and instruction
pattern = re.compile(r"(\S+)\s*<(\S+)>:\s*([\s\S]+?)\n\n", re.DOTALL)

# Parse the assembly into the desired format
functions = []

matches = pattern.findall(assembly)

# Function to remove addresses from instructions
def remove_addresses(lines):
    # Use regex to remove the address at the start of each instruction
    return [re.sub(r'^\S+\s+', '', line).strip() for line in lines]

for match in matches:
    address = match[0]
    function_name = match[1]
    instructions = match[2].strip().splitlines()

    # Clean the instructions to remove the address (addresses are the first part of each line)
    cleaned_instructions = remove_addresses(instructions)

    # Build the function string
    func_str = f"var {function_name} : List[String] = List("
    instruction_strs = [f"\"{line.strip()}\"" for line in cleaned_instructions]
    func_str += ", ".join(instruction_strs)
    func_str += ")"

    functions.append(func_str)

# Output the result
for func in functions:
    print(func)


var test_2 : List[String] = List("00200193           li    gp,2", "00000593           li    a1,0", "00000613           li    a2,0", "00c58733           add   a4,a1,a2", "00000393           li    t2,0", "4c771663           bne   a4,t2,8000066c <fail>")
var test_3 : List[String] = List("00300193           li    gp,3", "00100593           li    a1,1", "00100613           li    a2,1", "00c58733           add   a4,a1,a2", "00200393           li    t2,2", "4a771a63           bne   a4,t2,8000066c <fail>")
var test_4 : List[String] = List("00400193           li    gp,4", "00300593           li    a1,3", "00700613           li    a2,7", "00c58733           add   a4,a1,a2", "00a00393           li    t2,10", "48771e63           bne   a4,t2,8000066c <fail>")


In [6]:
import re

# Input assembly listing
assembly = """
8000018c <test_2>:
8000018c:	00200193          	li	gp,2
80000190:	00000593          	li	a1,0
80000194:	00000613          	li	a2,0
80000198:	40c58733          	sub	a4,a1,a2
8000019c:	00000393          	li	t2,0
800001a0:	4a771663          	bne	a4,t2,8000064c <fail>

800001a4 <test_3>:
800001a4:	00300193          	li	gp,3
800001a8:	00100593          	li	a1,1
800001ac:	00100613          	li	a2,1
800001b0:	40c58733          	sub	a4,a1,a2
800001b4:	00000393          	li	t2,0
800001b8:	48771a63          	bne	a4,t2,8000064c <fail>

800001bc <test_4>:
800001bc:	00400193          	li	gp,4
800001c0:	00300593          	li	a1,3
800001c4:	00700613          	li	a2,7
800001c8:	40c58733          	sub	a4,a1,a2
800001cc:	ffc00393          	li	t2,-4
800001d0:	46771e63          	bne	a4,t2,8000064c <fail>

800001d4 <test_5>:
800001d4:	00500193          	li	gp,5
800001d8:	00000593          	li	a1,0
800001dc:	ffff8637          	lui	a2,0xffff8
800001e0:	40c58733          	sub	a4,a1,a2
800001e4:	000083b7          	lui	t2,0x8
800001e8:	46771263          	bne	a4,t2,8000064c <fail>

800001ec <test_6>:
800001ec:	00600193          	li	gp,6
800001f0:	800005b7          	lui	a1,0x80000
800001f4:	00000613          	li	a2,0
800001f8:	40c58733          	sub	a4,a1,a2
800001fc:	800003b7          	lui	t2,0x80000
80000200:	44771663          	bne	a4,t2,8000064c <fail>

80000204 <test_7>:
80000204:	00700193          	li	gp,7
80000208:	800005b7          	lui	a1,0x80000
8000020c:	ffff8637          	lui	a2,0xffff8
80000210:	40c58733          	sub	a4,a1,a2
80000214:	800083b7          	lui	t2,0x80008
80000218:	42771a63          	bne	a4,t2,8000064c <fail>

8000021c <test_8>:
8000021c:	00800193          	li	gp,8
80000220:	00000593          	li	a1,0
80000224:	00008637          	lui	a2,0x8
80000228:	fff60613          	addi	a2,a2,-1 # 7fff <_start-0x7fff8001>
8000022c:	40c58733          	sub	a4,a1,a2
80000230:	ffff83b7          	lui	t2,0xffff8
80000234:	00138393          	addi	t2,t2,1 # ffff8001 <_end+0x7fff6001>
80000238:	40771a63          	bne	a4,t2,8000064c <fail>

8000023c <test_9>:
8000023c:	00900193          	li	gp,9
80000240:	800005b7          	lui	a1,0x80000
80000244:	fff58593          	addi	a1,a1,-1 # 7fffffff <_start-0x1>
80000248:	00000613          	li	a2,0
8000024c:	40c58733          	sub	a4,a1,a2
80000250:	800003b7          	lui	t2,0x80000
80000254:	fff38393          	addi	t2,t2,-1 # 7fffffff <_start-0x1>
80000258:	3e771a63          	bne	a4,t2,8000064c <fail>

8000025c <test_10>:
8000025c:	00a00193          	li	gp,10
80000260:	800005b7          	lui	a1,0x80000
80000264:	fff58593          	addi	a1,a1,-1 # 7fffffff <_start-0x1>
80000268:	00008637          	lui	a2,0x8
8000026c:	fff60613          	addi	a2,a2,-1 # 7fff <_start-0x7fff8001>
80000270:	40c58733          	sub	a4,a1,a2
80000274:	7fff83b7          	lui	t2,0x7fff8
80000278:	3c771a63          	bne	a4,t2,8000064c <fail>

8000027c <test_11>:
8000027c:	00b00193          	li	gp,11
80000280:	800005b7          	lui	a1,0x80000
80000284:	00008637          	lui	a2,0x8
80000288:	fff60613          	addi	a2,a2,-1 # 7fff <_start-0x7fff8001>
8000028c:	40c58733          	sub	a4,a1,a2
80000290:	7fff83b7          	lui	t2,0x7fff8
80000294:	00138393          	addi	t2,t2,1 # 7fff8001 <_start-0x7fff>
80000298:	3a771a63          	bne	a4,t2,8000064c <fail>

8000029c <test_12>:
8000029c:	00c00193          	li	gp,12
800002a0:	800005b7          	lui	a1,0x80000
800002a4:	fff58593          	addi	a1,a1,-1 # 7fffffff <_start-0x1>
800002a8:	ffff8637          	lui	a2,0xffff8
800002ac:	40c58733          	sub	a4,a1,a2
800002b0:	800083b7          	lui	t2,0x80008
800002b4:	fff38393          	addi	t2,t2,-1 # 80007fff <_end+0x5fff>
800002b8:	38771a63          	bne	a4,t2,8000064c <fail>

800002bc <test_13>:
800002bc:	00d00193          	li	gp,13
800002c0:	00000593          	li	a1,0
800002c4:	fff00613          	li	a2,-1
800002c8:	40c58733          	sub	a4,a1,a2
800002cc:	00100393          	li	t2,1
800002d0:	36771e63          	bne	a4,t2,8000064c <fail>

800002d4 <test_14>:
800002d4:	00e00193          	li	gp,14
800002d8:	fff00593          	li	a1,-1
800002dc:	00100613          	li	a2,1
800002e0:	40c58733          	sub	a4,a1,a2
800002e4:	ffe00393          	li	t2,-2
800002e8:	36771263          	bne	a4,t2,8000064c <fail>

800002ec <test_15>:
800002ec:	00f00193          	li	gp,15
800002f0:	fff00593          	li	a1,-1
800002f4:	fff00613          	li	a2,-1
800002f8:	40c58733          	sub	a4,a1,a2
800002fc:	00000393          	li	t2,0
80000300:	34771663          	bne	a4,t2,8000064c <fail>

80000304 <test_16>:
80000304:	01000193          	li	gp,16
80000308:	00d00593          	li	a1,13
8000030c:	00b00613          	li	a2,11
80000310:	40c585b3          	sub	a1,a1,a2
80000314:	00200393          	li	t2,2
80000318:	32759a63          	bne	a1,t2,8000064c <fail>

8000031c <test_17>:
8000031c:	01100193          	li	gp,17
80000320:	00e00593          	li	a1,14
80000324:	00b00613          	li	a2,11
80000328:	40c58633          	sub	a2,a1,a2
8000032c:	00300393          	li	t2,3
80000330:	30761e63          	bne	a2,t2,8000064c <fail>

80000334 <test_18>:
80000334:	01200193          	li	gp,18
80000338:	00d00593          	li	a1,13
8000033c:	40b585b3          	sub	a1,a1,a1
80000340:	00000393          	li	t2,0
80000344:	30759463          	bne	a1,t2,8000064c <fail>>
"""

# Regex pattern to match each function and instruction
pattern = re.compile(r"(\S+)\s*<(\S+)>:\s*([\s\S]+?)\n\n", re.DOTALL)

# Parse the assembly into the desired format
functions = []

matches = pattern.findall(assembly)

# Function to remove addresses and instruction codes from instructions
def remove_addresses_and_codes(lines):
    # Use regex to remove both the address and instruction code at the start of each line
    return [re.sub(r'^\S+\s+\S+\s+', '', line).strip() for line in lines]

for match in matches:
    address = match[0]
    function_name = match[1]
    instructions = match[2].strip().splitlines()

    # Clean the instructions to remove both the address and the instruction code
    cleaned_instructions = remove_addresses_and_codes(instructions)

    # Build the function string
    func_str = f"var {function_name} : ArrayBuffer[String] = ArrayBuffer("
    instruction_strs = [f"\"{line.strip()}\"" for line in cleaned_instructions]
    func_str += ", ".join(instruction_strs)
    func_str += ")"

    functions.append(func_str)

# Output the result
for func in functions:
    print(func)


var test_2 : ArrayBuffer[String] = ArrayBuffer("li	gp,2", "li	a1,0", "li	a2,0", "sub	a4,a1,a2", "li	t2,0", "bne	a4,t2,8000064c <fail>")
var test_3 : ArrayBuffer[String] = ArrayBuffer("li	gp,3", "li	a1,1", "li	a2,1", "sub	a4,a1,a2", "li	t2,0", "bne	a4,t2,8000064c <fail>")
var test_4 : ArrayBuffer[String] = ArrayBuffer("li	gp,4", "li	a1,3", "li	a2,7", "sub	a4,a1,a2", "li	t2,-4", "bne	a4,t2,8000064c <fail>")
var test_5 : ArrayBuffer[String] = ArrayBuffer("li	gp,5", "li	a1,0", "lui	a2,0xffff8", "sub	a4,a1,a2", "lui	t2,0x8", "bne	a4,t2,8000064c <fail>")
var test_6 : ArrayBuffer[String] = ArrayBuffer("li	gp,6", "lui	a1,0x80000", "li	a2,0", "sub	a4,a1,a2", "lui	t2,0x80000", "bne	a4,t2,8000064c <fail>")
var test_7 : ArrayBuffer[String] = ArrayBuffer("li	gp,7", "lui	a1,0x80000", "lui	a2,0xffff8", "sub	a4,a1,a2", "lui	t2,0x80008", "bne	a4,t2,8000064c <fail>")
var test_8 : ArrayBuffer[String] = ArrayBuffer("li	gp,8", "li	a1,0", "lui	a2,0x8", "addi	a2,a2,-1 # 7fff <_start-0x7fff8001>", "sub	a4,a1,